In [ ]:
import torch 
import torchvision
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets 
from torchvision.transforms import ToTensor
from matplotlib import pyplot as plt


In [ ]:
training_data = datasets.FashionMNIST(
    
    root = "data",
    train = True,
    download = True,
    transform = ToTensor()
)


Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw




Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw




Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw




Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw

Processing...
Done!


/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:502: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:143.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [ ]:
test_data = datasets.FashionMNIST(
    
    root = "data",
    train = False,
    download = True,
    transform = ToTensor()

)

In [ ]:
batch_size= 64
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

In [ ]:
for datapoint, label in train_dataloader:
  print("Shape of Datapoint [BatchSize, NumberChannels, h, w]",datapoint.shape)
  print(label)
  print(label.shape)
  

Shape of Datapoint [BatchSize, NumberChannels, h, w] torch.Size([64, 1, 28, 28])
tensor([9, 0, 0, 3, 0, 2, 7, 2, 5, 5, 0, 9, 5, 5, 7, 9, 1, 0, 6, 4, 3, 1, 4, 8,
        4, 3, 0, 2, 4, 4, 5, 3, 6, 6, 0, 8, 5, 2, 1, 6, 6, 7, 9, 5, 9, 2, 7, 3,
        0, 3, 3, 3, 7, 2, 2, 6, 6, 8, 3, 3, 5, 0, 5, 5])
torch.Size([64])
Shape of Datapoint [BatchSize, NumberChannels, h, w] torch.Size([64, 1, 28, 28])
tensor([0, 2, 0, 0, 4, 1, 3, 1, 6, 3, 1, 4, 4, 6, 1, 9, 1, 3, 5, 7, 9, 7, 1, 7,
        9, 9, 9, 3, 2, 9, 3, 6, 4, 1, 1, 8, 8, 0, 1, 1, 6, 8, 1, 9, 7, 8, 8, 9,
        6, 6, 3, 1, 5, 4, 6, 7, 5, 5, 9, 2, 2, 2, 7, 6])
torch.Size([64])
Shape of Datapoint [BatchSize, NumberChannels, h, w] torch.Size([64, 1, 28, 28])
tensor([4, 1, 8, 7, 7, 5, 4, 2, 9, 1, 7, 4, 6, 9, 7, 1, 8, 7, 1, 2, 8, 0, 9, 1,
        8, 7, 0, 5, 8, 6, 7, 2, 0, 8, 7, 1, 6, 2, 1, 9, 6, 0, 1, 0, 5, 5, 1, 7,
        0, 5, 8, 4, 0, 4, 0, 6, 6, 4, 0, 0, 4, 7, 3, 0])
torch.Size([64])
Shape of Datapoint [BatchSize, NumberChannels, h, w] to

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
print(device)

cuda


In [ ]:
class NeuralNetwork(nn.Module):
  def __init__(self):
    super(NeuralNetwork,self).__init__()
    self.flatten = nn.Flatten()

    self.linear_relu_stack = nn.Sequential(
        
        nn.Linear(in_features=28*28, out_features=512),
        nn.ReLU(),
        nn.Linear(in_features=512, out_features=512),
        nn.ReLU(),
        nn.Linear(in_features=512, out_features=10),
        nn.ReLU()
    )
  #X == Data
  def forward(self, data):
    data = self.flatten(data)

    logits = self.linear_relu_stack(data)

    return logits

In [ ]:
model = NeuralNetwork().to(device)



In [ ]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [ ]:
def train(dataloader, model, loss_fn, optimizer):
  size = len(dataloader.dataset)
  for i, (X, y) in enumerate(dataloader):
    X, y = X.to(device), y.to(device)

    pred = model(X)
    loss = loss_fn(pred, y)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if i % 100 == 0:
      loss, current = loss.item(), i*len(X)
      print(f"loss: {loss:>7f} [{current:>5d}/{size:>5d}]")


In [ ]:
epoch  = 15


for t in range(epoch):
  print(f"Epoch{t+1}\n--------------------")
  train(dataloader=train_dataloader, model= model, loss_fn=loss_fn, optimizer=optimizer)
print("done")

Epoch1
--------------------
loss: 2.301758 [    0/60000]
loss: 2.298744 [ 6400/60000]
loss: 2.290568 [12800/60000]
loss: 2.287221 [19200/60000]
loss: 2.282277 [25600/60000]
loss: 2.270247 [32000/60000]
loss: 2.270141 [38400/60000]
loss: 2.257324 [44800/60000]
loss: 2.261689 [51200/60000]
loss: 2.232514 [57600/60000]
Epoch2
--------------------
loss: 2.250498 [    0/60000]
loss: 2.251535 [ 6400/60000]
loss: 2.224568 [12800/60000]
loss: 2.215667 [19200/60000]
loss: 2.205114 [25600/60000]
loss: 2.193526 [32000/60000]
loss: 2.198427 [38400/60000]
loss: 2.176988 [44800/60000]
loss: 2.182891 [51200/60000]
loss: 2.132342 [57600/60000]
Epoch3
--------------------
loss: 2.179475 [    0/60000]
loss: 2.175067 [ 6400/60000]
loss: 2.125600 [12800/60000]
loss: 2.103360 [19200/60000]
loss: 2.076333 [25600/60000]
loss: 2.089327 [32000/60000]
loss: 2.088500 [38400/60000]
loss: 2.061403 [44800/60000]
loss: 2.070622 [51200/60000]
loss: 1.994090 [57600/60000]
Epoch4
--------------------
loss: 2.084274 [  